In [1]:
from pathlib import Path

from models.relational.db_schema import SQLite, Table, PostgreSQL, Database
from models.relational.config import DbConfig, DataBaseType
from models.relational.orm.sqlalchemy import SQLAlchemy
from models.relational.metadata import (
    ColumnMeta,
    ColumnType,
    ForeignKeyAction,
    ForeignKeyColumnMeta,
    UniqueColumnsMeta,
)

In [2]:
def get_postgresql_db() -> (
    PostgreSQL[
        SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
    ]
):
    return PostgreSQL[
        SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
    ](
        db_config=DbConfig(
            host="localhost",
            user="postgres",
            type=DataBaseType.POSTGRESQL,
            password="hugues",
            database="test",
        ),
        orm_class_=SQLAlchemy,
    )

In [3]:
def get_sqlite_db() -> (
    SQLite[SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn]
):
    db_path = Path.cwd() / "data" / "test.db"
    return SQLite[
        SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
    ](db_path, SQLAlchemy)

In [4]:
def get_table(
    database: Database[
        SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
    ],
    table_name: str,
    columns: list[ColumnMeta] | None = None,
    unique_constraints_columns: list[UniqueColumnsMeta] | None = None,
) -> Table[
    SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
]:
    """
    Récupérer une table de la base de données
    :param database: Base de données
    :param table_name: Nom de la table
    :param columns: Liste des colonnes de la table
    :param unique_constraints_columns: Liste des contraintes d'unicité
    """

    return Table[
        SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
    ](table_name, database, columns, unique_constraints_columns)

In [5]:
database = get_postgresql_db()
# database = get_sqlite_db()

In [6]:
schools_columns: list[ColumnMeta] = [
    ColumnMeta(
        name="Id",
        type=ColumnType.INT,
        length=None,
        nullable=False,
        primary_key=True,
        unique=True,
    ),
    ColumnMeta(
        name="Name",
        type=ColumnType.TEXT,
        length=100,
        nullable=False,
        primary_key=False,
        unique=False,
        default="ENSP Yaounde",
    ),
    ColumnMeta(
        name="Location",
        type=ColumnType.TEXT,
        length=100,
        nullable=False,
        primary_key=False,
        unique=False,
    ),
]

In [7]:
schools = get_table(database, "Schools", schools_columns)

In [8]:
for column in schools.columns:
    print(column)

Colonne Id de type INT de la table Schools
Colonne Name de type VARCHAR de la table Schools
Colonne Location de type VARCHAR de la table Schools


In [9]:
for column in schools.link_table.link_table.columns:
    print(column)

Schools.Id
Schools.Name
Schools.Location


In [10]:
students_columns: list[ColumnMeta] = [
    ColumnMeta(
        name="Id",
        type=ColumnType.INT,
        length=None,
        nullable=False,
        primary_key=True,
        unique=True,
    ),
    ColumnMeta(
        name="Name",
        type=ColumnType.TEXT,
        length=100,
        nullable=False,
        primary_key=False,
        unique=False,
    ),
    ForeignKeyColumnMeta(
        name="SchoolId",
        type=ColumnType.INT,
        length=None,
        nullable=False,
        primary_key=False,
        unique=False,
        foreign_table_name="Schools",
        foreign_column_name="Id",
        on_delete=ForeignKeyAction.CASCADE,
        on_update=ForeignKeyAction.CASCADE,
    ),
]

In [11]:
students = get_table(database, "Students", students_columns)

In [12]:
for column in students.columns:
    print(column)

Colonne Id de type INT de la table Students
Colonne Name de type VARCHAR de la table Students
Colonne SchoolId de type INT de la table Students avec clé étrangère vers la table Schools


In [13]:
schema_destination_path = Path.cwd() / "data" / "schema_postgresql.json"
# schema_destination_path = Path.cwd() / "data" / "schema_sqlite.json"

In [14]:
database.get_schema()

{'name': 'test',
 'type': 'postgresql',
 'tables': {'Schools': {'name': 'Schools',
   'columns': {'Id': {'name': 'Id',
     'type': 'INT',
     'length': None,
     'nullable': False,
     'primary_key': True,
     'unique': True,
     'default': None},
    'Name': {'name': 'Name',
     'type': 'VARCHAR',
     'length': 100,
     'nullable': False,
     'primary_key': False,
     'unique': False,
     'default': 'ENSP Yaounde'},
    'Location': {'name': 'Location',
     'type': 'VARCHAR',
     'length': 100,
     'nullable': False,
     'primary_key': False,
     'unique': False,
     'default': None}},
   'unique_columns': {}},
  'Students': {'name': 'Students',
   'columns': {'Id': {'name': 'Id',
     'type': 'INT',
     'length': None,
     'nullable': False,
     'primary_key': True,
     'unique': True,
     'default': None},
    'Name': {'name': 'Name',
     'type': 'VARCHAR',
     'length': 100,
     'nullable': False,
     'primary_key': False,
     'unique': False,
     'defau

In [15]:
database.save_schema(schema_destination_path)

In [16]:
new_school_column = ColumnMeta(
    name="Country",
    type=ColumnType.TEXT,
    length=100,
    nullable=False,
    primary_key=False,
    unique=False,
)

In [17]:
country_column = schools.add_column(new_school_column)

In [19]:
str(country_column)

'Colonne Country de type VARCHAR de la table Schools'